### Libs 

In [2]:
import pandas as pd
import openpyxl as xl
import schedule 
import time 
from datetime import datetime,timedelta

### Leitura Dataseat

In [3]:
def ler_excel(caminho_do_arquivo):
    n = 20000  # número de linhas
    try:
        # Lê o arquivo CSV
        df_read_excel = pd.read_csv(caminho_do_arquivo, encoding='ISO-8859-1', sep=';', nrows=n)
        # Exibe o conteúdo do DataFrame
        print(df_read_excel)  
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")  # Tratamento de erros



### Schedule para Ler o arquivo 

In [5]:
# Função de leitura para o agendamento
def agendar_leitura():
    caminho_do_arquivo = r'C:\Users\gm194\Downloads\votacao_secao_2024_RS.csv'
    ler_excel(caminho_do_arquivo)  # Chama a função que lê e imprime o arquivo

# Agendamento para executar a função agendar_leitura diariamente às 23:07
schedule.every().day.at("23:21").do(agendar_leitura)

# Loop para verificar se é hora de executar a tarefa agendada
while True:
    # Obtém a hora atual
    agora = datetime.now().strftime("%H:%M")
    
    # Verifica se o horário atual é igual ao horário agendado
    if agora == "23:21":  # Coloque aqui o horário que deseja
        agendar_leitura()  # Executa a função
        print("Leitura concluída. Encerrando o script.")
        break  # Sai do loop

    time.sleep(10)  # Espera 10 segundos antes de verificar novamente

       DT_GERACAO HH_GERACAO  ANO_ELEICAO  CD_TIPO_ELEICAO    NM_TIPO_ELEICAO  \
0      07/10/2024   13:30:12         2024                2  Eleição Ordinária   
1      07/10/2024   13:30:12         2024                2  Eleição Ordinária   
2      07/10/2024   13:30:12         2024                2  Eleição Ordinária   
3      07/10/2024   13:30:12         2024                2  Eleição Ordinária   
4      07/10/2024   13:30:12         2024                2  Eleição Ordinária   
...           ...        ...          ...              ...                ...   
19995  07/10/2024   13:30:12         2024                2  Eleição Ordinária   
19996  07/10/2024   13:30:12         2024                2  Eleição Ordinária   
19997  07/10/2024   13:30:12         2024                2  Eleição Ordinária   
19998  07/10/2024   13:30:12         2024                2  Eleição Ordinária   
19999  07/10/2024   13:30:12         2024                2  Eleição Ordinária   

       NR_TURNO  CD_ELEICAO